In [111]:
import pandas as pd
import openpyxl
import numpy as np
from pathlib import Path
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import re
import os
import sys

In [112]:
import pmp_functions_v5 as pmp

In [ ]:
# =============================================================================
# --- CONFIGURATION & CONSTANTS ---
# =============================================================================

# Paths & Targets
VOL_TARGET = 0.10
DATA_PATH = '../Data/'

# Strategy Parameters
MOMENTUM_LOOKBACKS = [1, 3, 12]  # Months for signal generation
SIGNAL_LAG = 1                   # Lag to apply to signal
K_SELECT = 3                     # Number of assets to select (CS Momentum)
MIN_ASSETS = 4                   # Min assets required to trade

# Risk Management & Costs
RF_ANNUAL = 0.02                 # Assumed risk-free rate (2%)
T_COST_BPS = 0.0000              # Transaction costs (10 basis points)
MAX_SCALAR_STRATEGY = 3.0        # Max leverage cap for individual asset class
MAX_SCALAR_PORTFOLIO = 3.0       # Max leverage cap for final aggregated portfolio

# Time & Frequency
ANNUALIZATION_FACTOR = 12
RESAMPLE_FREQ = "ME"             # Month End

# Derived Constants
RF_MONTHLY = RF_ANNUAL / ANNUALIZATION_FACTOR
T_COST = T_COST_BPS              # Passing directly as decimal if pmp expects decimal, or bps if expected bps

# =============================================================================
# 1. THE STRATEGY FACTORY
# =============================================================================
def run_independent_strategy(name, df_prices, benchmark, rf_series, k_select=K_SELECT, min_assets=MIN_ASSETS):
    """
    Runs a complete, isolated Momentum L/S strategy for a single asset class.
    """
    print(f"\n--- RUNNING STRATEGY: {name} ---")
    print(f"   Assets: {len(df_prices.columns)}")
    
    # A. Prepare Data
    returns = df_prices.pct_change().fillna(0)
    rf_series = rf_series.reindex(returns.index).fillna(RF_MONTHLY)
    
    # B. Signal Generation
    raw_signal = pd.DataFrame(0.0, index=df_prices.index, columns=df_prices.columns)
    
    for lag in MOMENTUM_LOOKBACKS:
        mom = df_prices.pct_change(lag)
        sig = np.sign(mom)
        raw_signal += sig
    
    final_signal = raw_signal / len(MOMENTUM_LOOKBACKS)
    
    # C. Weight Generation
    weights_unscaled = pmp.make_country_weights(
        signal=final_signal,
        returns=returns,
        benchmark_series=benchmark,
        k=k_select,            
        long_short=True,
        beta_neutral=False,    
        signal_lag=SIGNAL_LAG,
        min_regions=min_assets 
    )
    
    # D. Preliminary Backtest (Unscaled)
    # We use 0.0 cost here to get pure realized volatility for scaling
    res_unscaled = pmp.run_cc_strategy(
        weights=weights_unscaled,
        returns=returns,
        rf=rf_series,
        frequency=1,
        t_cost=0.0, 
        benchmark=benchmark
    )
    
    # E. Volatility Targeting
    ret_series = res_unscaled["ret_net"].dropna()
    realized_vol = ret_series.std() * np.sqrt(ANNUALIZATION_FACTOR)
    
    if realized_vol > 0:
        scalar = VOL_TARGET / realized_vol
        scalar = min(scalar, MAX_SCALAR_STRATEGY) 
    else:
        scalar = 1.0
        
    print(f"   Realized Vol (Unscaled): {realized_vol:.2%}")
    print(f"   Scaling Factor: {scalar:.2f}x")
    
    # F. Final Scaled Backtest
    weights_scaled = weights_unscaled * scalar
    res_final = pmp.run_cc_strategy(
        weights=weights_scaled,
        returns=returns,
        rf=rf_series,
        frequency=1,
        t_cost=T_COST, 
        benchmark=benchmark
    )
    
    return res_final

# =============================================================================
# 2. DATA LOADING & PREPROCESSING
# =============================================================================

def load_rf_csv(path: Path) -> pd.Series:
    """
    Load risk-free rate from CSV. 
    Format: Date, RF (returns)
    """
    df = pd.read_csv(path)
    
    # Parse dates and align to MonthEnd
    df["Date"] = pd.to_datetime(df["Date"]) + MonthEnd(0)
    
    df = df.set_index("Date").sort_index()
    
    # Return the RF column
    return df["RF"]

def load_and_process(filename, suffix, is_fx=False):
    """
    Helper to load and process data to avoid repetition.
    """
    print(f"Loading {suffix}...")
    full_path = os.path.join(DATA_PATH, filename)
    
    # Handle FX Returns vs Price data structure
    if is_fx:
        df = pd.read_excel(full_path, sheet_name="RETURNS", engine='openpyxl')
    else:
        df = pd.read_excel(full_path, engine='openpyxl')
        
    # Index setting
    if 'Date' in df.columns:
        df = df.set_index('Date')
    elif 'Dates' in df.columns:
        df = df.set_index('Dates')
    else:
        df = df.set_index(df.columns[0])
    
    df.index = pd.to_datetime(df.index, errors='coerce')
    
    # FX special handling
    if is_fx:
        df = (1 + df.fillna(0)).cumprod() * 100
    
    # Renaming and resampling
    df.columns = [f"{c}{suffix}" for c in df.columns]
    df = df.apply(pd.to_numeric, errors='coerce').resample(RESAMPLE_FREQ).last().ffill()
    
    # Ensure data goes up to 2025-11-30
    target_date = pd.Timestamp("2025-11-30")
    if df.index[-1] != target_date:
        last_row = df.iloc[[-1]].copy()
        last_row.index = [target_date]
        df = pd.concat([df, last_row]).sort_index()
    
    return df

# --- Load Assets ---
df_eq = load_and_process("Equity Data.xlsx", "")
df_bd = load_and_process("Bond Data.xlsx", "")
df_rt = load_and_process("Interest Rates Data.xlsx", "")
df_fx = load_and_process("FX Data.xlsx", "", is_fx=True)

rf_series = load_rf_csv(os.path.join(DATA_PATH, "RF.csv"))

# --- Load Benchmark ---
df_bench = pd.read_excel(os.path.join(DATA_PATH, "Benchmarks.xlsx"), engine='openpyxl')
if 'Date' in df_bench.columns: df_bench = df_bench.set_index('Date')
else: df_bench = df_bench.set_index(df_bench.columns[0])
df_bench.index = pd.to_datetime(df_bench.index, errors='coerce')
df_bench = df_bench.resample(RESAMPLE_FREQ).last().ffill()
benchmark_ret = df_bench['Full Benchmark'].pct_change().fillna(0)

# =============================================================================
# 3. EXECUTION PER ASSET CLASS
# =============================================================================

ret_equity = run_independent_strategy("EQUITY", df_eq, benchmark=benchmark_ret, rf_series=rf_series)
ret_bond = run_independent_strategy("BOND", df_bd, benchmark=benchmark_ret, rf_series=rf_series)
ret_rates = run_independent_strategy("RATES", df_rt, benchmark=benchmark_ret, rf_series=rf_series)
ret_fx = run_independent_strategy("FX", df_fx, benchmark=benchmark_ret, rf_series=rf_series)

# =============================================================================
# 4. AGGREGATION & PORTFOLIO CONSTRUCTION
# =============================================================================

# Combine Strategies
df_strategies = pd.DataFrame({
    'Equity': ret_equity["ret_net"],
    'Bond': ret_bond["ret_net"],
    'Rates': ret_rates["ret_net"],
    'FX': ret_fx["ret_net"]
}).dropna()

# Portfolio Construction (Equal Weight across strategies)
final_portfolio_ret = df_strategies.mean(axis=1)

# Re-Scale to Target Vol
port_vol = final_portfolio_ret.std() * np.sqrt(ANNUALIZATION_FACTOR)
final_scalar = VOL_TARGET / port_vol if port_vol > 0 else 1.0
final_scalar = min(final_scalar, MAX_SCALAR_PORTFOLIO)
final_portfolio_ret = final_portfolio_ret * final_scalar

print(f"\nFinal Portfolio Re-Scaling Factor: {final_scalar:.2f}x")

# Create Consolidated Dataframe for Summary
df_all = df_strategies.copy()
df_all['Total_Portfolio'] = final_portfolio_ret

# Align everything
common_idx = df_all.index.intersection(benchmark_ret.index)
df_all = df_all.loc[common_idx]
bm_aligned = benchmark_ret.loc[common_idx]
rf_aligned = pd.Series(RF_MONTHLY, index=common_idx)

# Calculate Excess Returns
xs_all = df_all.sub(rf_aligned, axis=0)
xs_bench = (bm_aligned - rf_aligned).to_frame()

# =============================================================================
# 5. SUMMARY
# =============================================================================

print("\n=======================================================")
print("             PERFORMANCE SUMMARY PER ASSET CLASS       ")
print("=======================================================")

summary = pmp.summarizePerformance(
    xsReturns=xs_all.values,
    Rf=rf_aligned.values.reshape(-1, 1),
    factorXsReturns=xs_bench.values, 
    annualizationFactor=ANNUALIZATION_FACTOR,
    strategyNames=df_all.columns.tolist()
)

print(summary)

# Correlation Matrix
print("\n--- Correlation Matrix ---")
print(df_all.corr().round(2))

# =============================================================================
# 6. VISUALIZATION
# =============================================================================
cum_all = (1 + df_all).cumprod()
cum_bench = (1 + xs_bench).cumprod()

plt.figure(figsize=(12, 8))
colors = ['cyan', 'lightgreen', 'orange', 'pink']

# Plot Components
fig, ax = plt.subplots(figsize=(12, 6))

# Component strategies (light, semi-transparent)
for i, col in enumerate(df_strategies.columns):
    ax.plot(
        cum_all[col],
        label=f"{col} L/S",
        linewidth=1.2,
        alpha=0.35,
        color=colors[i],
    )

# Total Portfolio (strong highlight)
ax.plot(
    cum_all['Total_Portfolio'],
    label=f"Total Portfolio ({VOL_TARGET:.0%} Vol)",
    linewidth=2.8,
    color="blue",
)

# Benchmark (clean dashed)
ax.plot(
    cum_bench,
    label="Benchmark (60% MSCI World / 40% FTSE World Govt Bond Index)",
    linewidth=2,
    linestyle="--",
    color="black",
    alpha=0.8,
)

# Zero line for excess returns (important)
ax.axhline(0, color="gray", linewidth=1, alpha=0.4)

# Labels & title
ax.set_title("Multi-Asset Risk Parity – Excess Return Breakdown", pad=12)
ax.set_ylabel("Cumulative Excess Return")
ax.set_xlabel("Date")

# Legend
ax.legend(frameon=False, ncol=2)

# Grid
ax.grid(True, which="both", alpha=0.25)

plt.tight_layout()
plt.show()

Loading ...
Loading ...
Loading ...
Loading ...


## Equity Table

In [119]:
#ret_equity.to_csv("../Results/trend_equity.csv")
ret_equity.loc["2005-12-31":"2007-11-30"]

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_AU,w_CH,w_JP,w_UK,w_EM,w_EU
Date,,,,,,,,,,,,,
2005-12-31,0.025798,0.025798,0.017077,0.787261,0.0,0.004208,-0.265825,-0.265825,0.354433,0.354433,0.044304,0.044304,-0.265825
2006-01-31,-0.000068,-0.000068,0.029591,0.578181,0.0,0.004008,-0.265825,-0.265825,0.478484,0.106330,-0.265825,0.106330,0.106330
2006-02-28,-0.002693,-0.002693,-0.002953,0.349153,0.0,0.003807,-0.265825,-0.265825,0.265825,0.265825,-0.265825,0.000000,0.265825
2006-03-31,0.009146,0.009146,0.006510,0.659745,0.0,0.004610,-0.265825,-0.265825,-0.265825,0.199368,0.199368,0.199368,0.199368
2006-04-30,-0.016351,-0.016351,0.027651,0.014516,0.0,0.003807,-0.265825,-0.265825,-0.265825,0.199368,0.199368,0.199368,0.199368
2006-05-31,0.008754,0.008754,-0.009579,0.033110,0.0,0.004409,-0.265825,-0.265825,-0.265825,0.199368,0.199368,0.199368,0.199368
2006-06-30,-0.001348,-0.001348,-0.004859,0.028514,0.0,0.004409,-0.265825,-0.265825,-0.265825,0.199368,0.199368,0.199368,0.199368
2006-07-31,0.021841,0.021841,0.007211,0.775172,0.0,0.004008,-0.265825,-0.265825,0.354433,-0.265825,0.354433,0.044304,0.044304
2006-08-31,0.003418,0.003418,0.018441,0.766861,0.0,0.004610,-0.265825,0.265825,-0.265825,-0.265825,0.265825,0.000000,0.265825


## Bond Table

In [ ]:
ret_bond.to_csv("../Results/trend_bond.csv")
ret_bond

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_EU,w_JP,w_AU,w_US,w_CH,w_EM,w_UK
Date,,,,,,,,,,,,,
1999-03-31,-0.008123,-0.008123,0.000000,0.000000,0.0,0.004610,-0.586366,-0.586366,-0.586366,0.781821,0.781821,0.097728,0.097728
1999-04-30,0.029696,0.029696,0.000000,1.382731,0.0,0.004208,-0.586366,0.781821,-0.586366,-0.586366,0.781821,0.097728,0.097728
1999-05-31,0.025884,0.025884,0.000000,0.547426,0.0,0.004008,-0.586366,1.055459,-0.586366,-0.586366,0.234546,0.234546,0.234546
1999-06-30,0.010155,0.010155,0.000000,1.528064,0.0,0.004409,-0.586366,0.586366,-0.586366,0.586366,0.586366,0.000000,-0.586366
1999-07-31,0.004043,0.004043,0.000000,1.192627,0.0,0.004208,-0.586366,0.781821,-0.586366,-0.586366,0.781821,0.097728,0.097728
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.018137,0.018137,0.038273,1.531227,0.0,0.004008,0.586366,-0.586366,-0.586366,0.586366,0.586366,-0.586366,0.000000
2025-07-31,0.002840,0.002840,0.006737,2.360963,0.0,0.004409,-0.586366,-0.586366,0.097728,-0.586366,0.781821,0.781821,0.097728
2025-08-31,0.004519,0.004519,0.023840,1.380583,0.0,0.001667,-0.586366,-0.586366,-0.586366,0.781821,0.781821,0.097728,0.097728


## Rate Table

In [116]:
ret_rates.to_csv("../Results/trend_rates.csv")
ret_rates

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_CH,w_EU,w_AU,w_US,w_EM,w_UK,w_JP
Date,,,,,,,,,,,,,
1999-04-30,0.018318,0.018318,0.000000,0.000000,0.0,0.004208,1.5,0.00,-1.5,-1.5,1.50,0.00,0.00
1999-05-31,-0.000860,-0.000860,0.000000,4.258852,0.0,0.004008,-1.5,0.25,2.0,-1.5,0.25,0.25,0.25
1999-06-30,-0.010152,-0.010152,0.000000,4.249413,0.0,0.004409,1.5,0.00,-1.5,-1.5,1.50,0.00,0.00
1999-07-31,-0.008355,-0.008355,0.000000,3.003564,0.0,0.004208,-1.5,0.00,0.0,-1.5,1.50,0.00,1.50
1999-08-31,0.000027,0.000027,0.000000,2.499813,0.0,0.004409,-1.5,0.25,2.0,-1.5,0.25,0.25,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.006585,0.006585,0.038273,6.010986,0.0,0.004008,-1.0,1.00,1.0,-1.0,0.00,1.00,-1.00
2025-07-31,-0.003300,-0.003300,0.006737,1.003394,0.0,0.004409,-1.0,1.00,1.0,-1.0,1.00,0.00,-1.00
2025-08-31,-0.003431,-0.003431,0.023840,1.001650,0.0,0.001667,-1.0,1.00,1.0,-1.0,0.00,1.00,-1.00


## FX Table

In [117]:
ret_fx.to_csv("../Results/trend_fx.csv")
ret_fx

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_CH,w_EU,w_JP,w_UK,w_AU,w_EM
Date,,,,,,,,,,,,
1990-03-31,0.019121,0.019121,0.000000,0.000000,0.0,0.006621,0.714245,-0.510175,-0.510175,0.714245,-0.510175,0.102035
1990-04-30,0.027526,0.027526,0.000000,0.020327,0.0,0.006017,0.714245,-0.510175,-0.510175,0.714245,-0.510175,0.102035
1990-05-31,-0.005865,-0.005865,0.000000,0.015053,0.0,0.006621,0.714245,-0.510175,-0.510175,0.714245,-0.510175,0.102035
1990-06-30,0.051674,0.051674,0.000000,1.052621,0.0,0.006319,0.510175,-0.510175,-0.510175,0.510175,0.510175,-0.510175
1990-07-31,0.046123,0.046123,0.000000,0.026462,0.0,0.006319,0.510175,-0.510175,-0.510175,0.510175,0.510175,-0.510175
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.004756,-0.004756,0.038273,2.452557,0.0,0.004008,-0.510175,0.714245,0.714245,0.102035,-0.510175,-0.510175
2025-07-31,0.005925,0.005925,0.006737,2.475064,0.0,0.004409,0.714245,-0.510175,-0.510175,0.714245,-0.510175,0.102035
2025-08-31,0.003043,0.003043,0.023840,1.006137,0.0,0.001667,0.510175,0.510175,-0.510175,0.510175,-0.510175,-0.510175
